# **ルワンダにおける CO2 排出量を予測する**
Predict CO2 Emissions in Rwanda(https://www.kaggle.com/competitions/playground-series-s3e20)

# 参考資料
**書籍**


**Kaggle**
- [Getting Started: Modeling](https://www.kaggle.com/code/inversion/getting-started-modeling)


**自分で作成したファイル**
- Home_Credit_Default_Risk
- 肝硬変の転帰の多クラス予測
- 馬の健康への影響を予測する

**その他**


## ライブラリ・データセットのインポート

In [113]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [114]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor # 特徴量の重要度評価で用いる

## 実行時間を調べるために使う

In [115]:
import datetime
import time
import math

start_time = time.time()

def changeHMS(s):
    h = math.floor(s / 3600)
    if h > 0:
        s = s - h * 3600
        indi_h = str(h) + 'h'
    else:
        indi_h = ''
    m = math.floor(s / 60)
    if m > 0:
        indi_m = str(m) + 'm'
    else:
        indi_m = ''
    s = math.floor(s % 60)
    time = indi_h + indi_m + str(s) + 's'
    return time

In [116]:
FILE_PATH = ''
OUTPUT_DIR = ''

train = pd.read_csv(FILE_PATH + 'train.csv')
test = pd.read_csv(FILE_PATH + 'test.csv')

# train_id = train['id']

# test_id = test['id']

target = train['emission']
target_col = 'emission'

target_name = str(train.iloc[:, [-1]].columns.tolist()) # カラム数-2の値が目的変数

df = pd.concat([train, test], axis=0)
df.reset_index(inplace=True)

# 説明変数をデータ型ごとに代入する
numerical_features = df._get_numeric_data().columns
categorical_features = df.drop(numerical_features, axis=1).columns

In [117]:
df.head(10)

,index,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,CarbonMonoxide_H2O_column_number_density,CarbonMonoxide_cloud_height,CarbonMonoxide_sensor_altitude,CarbonMonoxide_sensor_azimuth_angle,CarbonMonoxide_sensor_zenith_angle,CarbonMonoxide_solar_azimuth_angle,CarbonMonoxide_solar_zenith_angle,NitrogenDioxide_NO2_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,NitrogenDioxide_stratospheric_NO2_column_number_density,NitrogenDioxide_NO2_slant_column_number_density,NitrogenDioxide_tropopause_pressure,NitrogenDioxide_absorbing_aerosol_index,NitrogenDioxide_cloud_fraction,NitrogenDioxide_sensor_altitude,NitrogenDioxide_sensor_azimuth_angle,NitrogenDioxide_sensor_zenith_angle,NitrogenDioxide_solar_azimuth_angle,NitrogenDioxide_solar_zenith_angle,Formaldehyde_tropospheric_HCHO_column_number_density,Formaldehyde_tropospheric_HCHO_column_number_density_amf,Formaldehyde_HCHO_slant_column_number_density,Formaldehyde_cloud_fraction,Formaldehyde_solar_zenith_angle,Formaldehyde_solar_azimuth_angle,Formaldehyde_sensor_zenith_angle,Formaldehyde_sensor_azimuth_angle,UvAerosolIndex_absorbing_aerosol_index,UvAerosolIndex_sensor_altitude,UvAerosolIndex_sensor_azimuth_angle,UvAerosolIndex_sensor_zenith_angle,UvAerosolIndex_solar_azimuth_angle,UvAerosolIndex_solar_zenith_angle,Ozone_O3_column_number_density,Ozone_O3_column_number_density_amf,Ozone_O3_slant_column_number_density,Ozone_O3_effective_temperature,Ozone_cloud_fraction,Ozone_sensor_azimuth_angle,Ozone_sensor_zenith_angle,Ozone_solar_azimuth_angle,Ozone_solar_zenith_angle,UvAerosolLayerHeight_aerosol_height,UvAerosolLayerHeight_aerosol_pressure,UvAerosolLayerHeight_aerosol_optical_depth,UvAerosolLayerHeight_sensor_zenith_angle,UvAerosolLayerHeight_sensor_azimuth_angle,UvAerosolLayerHeight_solar_azimuth_angle,UvAerosolLayerHeight_solar_zenith_angle,Cloud_cloud_fraction,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,0,ID_-0.510_29.290_2019_00,-0.51000,29.29000,2019,0,-0.00011,0.60302,-0.00007,0.25567,-98.59389,50.84356,-130.05080,35.87450,-0.00003,0.03537,1589.02454,4061.09814,829530.50000,71.11198,52.77593,-149.87556,25.96521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00012,0.86323,0.00004,0.25567,35.87450,-130.05080,50.84356,-98.59389,-1.28076,829864.54688,-12.62898,35.63242,-138.78645,30.75213,0.11593,2.50661,0.29566,225.73114,0.59547,-12.62898,35.63242,-138.78645,30.75213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.59547,53534.73242,3664.43622,61085.80957,2615.12048,15.56853,0.27229,-12.62899,35.63242,-138.78642,30.75214,3.75099
1,1,ID_-0.510_29.290_2019_01,-0.51000,29.29000,2019,1,0.00002,0.72821,0.00001,0.13099,16.59286,39.13719,-140.87443,28.96513,0.00001,0.03653,1772.57441,1869.04041,829787.28713,-1.01959,38.98237,-140.15805,29.56200,0.00005,0.00002,0.00003,0.00009,7311.86914,-1.93539,0.06704,829859.96037,5.47104,35.26519,-138.34391,30.05426,0.00017,1.17283,0.00014,0.20075,29.07178,-141.81483,43.05021,4.67884,-1.54812,829747.85697,16.15249,43.48533,-142.78614,28.57363,0.11678,2.65770,0.31573,226.17217,0.17517,24.46434,42.59654,-143.09787,28.21366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21361,63790.29624,3651.19031,66969.47873,3174.57242,8.69060,0.25683,30.35938,39.55763,-145.18393,27.25178,4.02518
2,2,ID_-0.510_29.290_2019_02,-0.51000,29.29000,2019,2,0.00051,0.74820,0.00038,0.11002,72.79584,52.86882,-150.19176,23.20642,0.00015,0.03534,2703.23680,2809.13839,829883.82869,-54.801

## 前処理

### 欠損値の補完

In [118]:
def missing_values(df):
    df = df.drop(['location'], axis=1).fillna(0)

    return df

### 外れ値の修正

In [119]:
def outlier(df):

    return df

### スケーリング（標準化）

- MinMaxScaler(正規化)

In [120]:
def scaling_MinMaxScaler(df):
    df_scale = df[numerical_features]
    sc = MinMaxScaler()
    df[numerical_features] = pd.DataFrame(sc.fit_transform(df_scale), columns = df_scale.columns)

    return df

## 特徴量エンジニアリング

### 特徴量の作成

In [121]:
def create_new_features(df):
    # 緯度経度ごとの平均
    df['location'] = [str(x) + '_' + str(y) for x, y in zip(df.latitude, df.longitude)]

    # 周期ごとの平均
    df_roll_mean = df.sort_values(by=['location', 'year', 'week_no']).groupby(['location'])[df.columns[5:].tolist()].rolling(window=2).mean().reset_index()
    df_roll_mean.drop(['level_1', 'emission', 'location'], axis=1, inplace=True)
    df_roll_mean.columns = [col + '_roll_mean' for col in df_roll_mean.columns]

    df = df.sort_values(by=['location', 'year', 'week_no'], ignore_index=True).merge(df_roll_mean, how='left', left_index=True, right_index=True)

    return df

### カテゴリ変数のエンコーディング

* One-HotEncoding

In [122]:
def one_hot_encoding(df, cat_cols):
    df = pd.get_dummies(df, columns=cat_cols)

    return df

- LabelEncoding

In [123]:
def label_encoder(df):
    categorical_columns = df.select_dtypes(include=['object']).columns

    for column in categorical_columns:
        df[column] = df[column].fillna('').astype('str') # 欠損値の補完をする
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])

    return df

### 特徴量の選択

#### 特徴量の重要度評価

In [124]:
def feature_importance_evaluation(df):
    # 訓練データをX(説明変数)とy（目的変数）に分割する
    X = df.select_dtypes(include=['float', 'int'])
    X = X.drop(['emission'], axis=1) # 目的変数を指定する
    y = target # 目的変数を指定する

    # for column in X.columns.tolist():
    #     X[column] = X[column].apply(lambda x: np.log(x + 1))

    # 特徴量の重要度評価
    clf = RandomForestRegressor(
        random_state=42,
    )

    clf.fit(X, y)
    importance = clf.feature_importances_

    feature_importance = pd.DataFrame(data=importance, index=X.columns, columns=['importance']) \
        .sort_values(ascending=True, by='importance')

    return feature_importance

#### 特徴量の削除

In [125]:
def drop_columns(df):
    drop_list = [

    ]
    dropped_df = df.drop(columns=drop_list)

    return dropped_df

### データセットの更新

In [126]:
# 前処理
# df = missing_values(df) # 今回は特徴量エンジニアリングのあとに指定する
df = outlier(df)
# df = scaling_MinMaxScaler(df) # 標準化したい時のみ実行する

# 特徴量エンジニアリング
df = create_new_features(df)
df = missing_values(df) # 今回はここに設定した
df = drop_columns(df)

ohe_cols = [] # One-Hot Encodingしたい水準数の少ないカラムを指定する
df = one_hot_encoding(df, ohe_cols)
df = label_encoder(df)

In [127]:
train = df[:79023]
test = df[79023:]

train_x = train.drop(columns=['emission'])
train_y = target
test_x = test.drop(columns=['emission'])

X = train_x.values
y = train_y.values
# y = y.astype(int)

df.head()

,index,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,CarbonMonoxide_H2O_column_number_density,CarbonMonoxide_cloud_height,CarbonMonoxide_sensor_altitude,CarbonMonoxide_sensor_azimuth_angle,CarbonMonoxide_sensor_zenith_angle,CarbonMonoxide_solar_azimuth_angle,CarbonMonoxide_solar_zenith_angle,NitrogenDioxide_NO2_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,NitrogenDioxide_stratospheric_NO2_column_number_density,NitrogenDioxide_NO2_slant_column_number_density,NitrogenDioxide_tropopause_pressure,NitrogenDioxide_absorbing_aerosol_index,NitrogenDioxide_cloud_fraction,NitrogenDioxide_sensor_altitude,NitrogenDioxide_sensor_azimuth_angle,NitrogenDioxide_sensor_zenith_angle,NitrogenDioxide_solar_azimuth_angle,NitrogenDioxide_solar_zenith_angle,Formaldehyde_tropospheric_HCHO_column_number_density,Formaldehyde_tropospheric_HCHO_column_number_density_amf,Formaldehyde_HCHO_slant_column_number_density,Formaldehyde_cloud_fraction,Formaldehyde_solar_zenith_angle,Formaldehyde_solar_azimuth_angle,Formaldehyde_sensor_zenith_angle,Formaldehyde_sensor_azimuth_angle,UvAerosolIndex_absorbing_aerosol_index,UvAerosolIndex_sensor_altitude,UvAerosolIndex_sensor_azimuth_angle,UvAerosolIndex_sensor_zenith_angle,UvAerosolIndex_solar_azimuth_angle,UvAerosolIndex_solar_zenith_angle,Ozone_O3_column_number_density,Ozone_O3_column_number_density_amf,Ozone_O3_slant_column_number_density,Ozone_O3_effective_temperature,Ozone_cloud_fraction,Ozone_sensor_azimuth_angle,Ozone_sensor_zenith_angle,Ozone_solar_azimuth_angle,Ozone_solar_zenith_angle,UvAerosolLayerHeight_aerosol_height,UvAerosolLayerHeight_aerosol_pressure,UvAerosolLayerHeight_aerosol_optical_depth,UvAerosolLayerHeight_sensor_zenith_angle,UvAerosolLayerHeight_sensor_azimuth_angle,UvAerosolLayerHeight_solar_azimuth_angle,UvAerosolLayerHeight_solar_zenith_angle,Cloud_cloud_fraction,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission,week_no_roll_mean,SulphurDioxide_SO2_column_number_density_roll_mean,SulphurDioxide_SO2_column_number_density_amf_roll_mean,SulphurDioxide_SO2_slant_column_number_density_roll_mean,SulphurDioxide_cloud_fraction_roll_mean,SulphurDioxide_sensor_azimuth_angle_roll_mean,SulphurDioxide_sensor_zenith_angle_roll_mean,SulphurDioxide_solar_azimuth_angle_roll_mean,SulphurDioxide_solar_zenith_angle_roll_mean,SulphurDioxide_SO2_column_number_density_15km_roll_mean,CarbonMonoxide_CO_column_number_density_roll_mean,CarbonMonoxide_H2O_column_number_density_roll_mean,CarbonMonoxide_cloud_height_roll_mean,CarbonMonoxide_sensor_altitude_roll_mean,CarbonMonoxide_sensor_azimuth_angle_roll_mean,CarbonMonoxide_sensor_zenith_angle_roll_mean,CarbonMonoxide_solar_azimuth_angle_roll_mean,CarbonMonoxide_solar_zenith_angle_roll_mean,NitrogenDioxide_NO2_column_number_density_roll_mean,NitrogenDioxide_tropospheric_NO2_column_number_density_roll_mean,NitrogenDioxide_stratospheric_NO2_column_number_density_roll_mean,NitrogenDioxide_NO2_slant_column_number_density_roll_mean,NitrogenDioxide_tropopause_pressure_roll_mean,NitrogenDioxide_absorbing_aerosol_index_roll_mean,NitrogenDioxide_cloud_fraction_roll_mean,NitrogenDioxide_sensor_altitude_roll_mean,NitrogenDioxide_sensor_azimuth_angle_roll_mean,NitrogenDioxide_sensor_zenith_angle_roll_mean,NitrogenDioxide_solar_azimuth_angle_roll_mean,NitrogenDioxide_solar_zenith_angle_roll_mean,Formaldehyde_tropospheric_HCHO_column_number_density_roll_mea

In [128]:
# # 特徴量の重要度評価
# train_importance = train
# feature_importance = feature_importance_evaluation(train_importance)
# feature_importance

In [129]:
# feature_importance.plot(kind='barh', figsize=(10, 15), color='b')

In [130]:
df.to_csv(OUTPUT_DIR + 'data.csv', index=False)

In [131]:
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    train = data[:79023].reset_index(drop=True)
    test = data[79023:].reset_index(drop=True).drop('emission', axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop(['emission', 'ID_LAT_LON_YEAR_WEEK'], axis=1) # 目的変数を指定する
    y = train['emission'].values # 目的変数を指定する

    return data,test,train,X,y

In [132]:
filename = 'data.csv'
data,test,train,X,y = file_to_xy(OUTPUT_DIR + filename)

読み込み完了 data.csv
